In [1]:
# Aqui usamos somente o conjunto de treinamento para selecionar as melhores features com Mutual Info

from utils import  *
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import defaultdict
from pandas import DataFrame
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

from sklearn.model_selection import train_test_split
import random 

# 1) Separar o corpus por classes
numRemover = NumRemover()
all_data = numRemover.fit_transform(get_data_from_db())
seed = int(random.uniform(0, 100))
train_ratio = 0.7
train, test, ytrain, ytest = train_test_split(all_data, all_data["labels"], train_size = train_ratio, stratify = all_data["labels"], random_state = seed)

# 2) Criar um vetorizer para unigrams
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words= stopwords.words("portuguese"), strip_accents= "unicode")

# 3) Fazer o fit para todo o corpus
X = vectorizer.fit_transform(all_data["texts"])

# 4) Aplicar o metodo de seleção
dataX = SelectKBest(mutual_info_classif, k = 8000).fit_transform(X.toarray(), all_data["labels"])


In [2]:
sKFold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= True)
print("Cross Validation:")
accuracy_average = np.array([])
dataY = all_data["labels"]
classifier = MultinomialNB()
for index, (train, test) in enumerate(sKFold.split(dataX, dataY)):
    # Treinando um modelo Naive Bayes
    trainX = dataX[train]
    testX = dataX[test]

    trainY = dataY[train]
    testY = dataY[test]

    accuracy = train_classifier2(trainX, trainY, testX, testY, classifier)
    # classifier_models.append(pipeline)

    accuracy_average = np.append(accuracy_average, accuracy)
    print("Fold ", index, " - Acuracia: ", accuracy)

print("Accuracia media: ", accuracy_average.mean())
print("Desvio padrão: ", accuracy_average.std())

Cross Validation:
Fold  0  - Acuracia:  0.556603773585
Fold  1  - Acuracia:  0.567307692308
Fold  2  - Acuracia:  0.605769230769
Fold  3  - Acuracia:  0.576923076923
Fold  4  - Acuracia:  0.634615384615
Fold  5  - Acuracia:  0.653846153846
Fold  6  - Acuracia:  0.567307692308
Fold  7  - Acuracia:  0.557692307692
Fold  8  - Acuracia:  0.615384615385
Fold  9  - Acuracia:  0.557692307692
Accuracia media:  0.589314223512
Desvio padrão:  0.0337030826868
